# Aggregators

Using `Recorder` to see the recipes.

In [ ]:
import grblas as gb
from grblas import Recorder, Matrix, Vector, agg

In [ ]:
# We can create a recorder and have it be alive until we call `rec.stop()`.
# One can do `rec.clear()` to clear the data.
rec = Recorder()
A = Matrix.from_values(
    [3, 0, 3, 5, 6, 0, 6, 1, 6, 2, 5],
    [0, 1, 2, 2, 2, 3, 3, 4, 4, 5, 5],
    [3, 2, 3, 1, 5, 3, 7, 8, 3, 1, 7],
    dtype="int32",
    name="A",
)
A

In [ ]:
rec

In [ ]:
# You can also record within a context.
# Only one recorder records at a given time, so `rec` does not save these calls
with Recorder() as rec2:
    result = A.reduce_rows(agg.min).new()
result

In [ ]:
# Here we see our recipe simply used a monoid as one would expect for MIN
rec2

In [ ]:
rec.clear()
A.reduce_columns(agg.count).new()

### Look at recipes for all the aggregators

In [ ]:
from IPython.display import display, HTML, Markdown

In [ ]:
skipped = set()
for aggname in sorted(dir(agg)):
    aggregator = getattr(agg, aggname)
    if not isinstance(aggregator, agg.Aggregator):
        continue
    try:
        with Recorder() as r:
            # The type-handing of aggregators still needs work, so hack it for now
            if aggname in {"all", "any"}:
                B = A.dup(dtype=bool)
            elif aggname in {"bitwise_all", "bitwise_any"}:
                B = A.dup(dtype="UINT64")
            else:
                B = A
            result = B.reduce_rows(aggregator).new()
            # result = B.reduce_columns(aggregator).new()
            # result = B.reduce_scalar(aggregator).new()
            # result = B.T.reduce_rows(aggregator).new()
            # result = B.T.reduce_columns(aggregator).new()
            # result = B.T.reduce_scalar(aggregator).new()
            display(Markdown(f"# {aggname}"))
            display(HTML(result._repr_html_()))
            display(Markdown(r._repr_markdown_()))
    except Exception:
        skipped.add(aggname)

In [ ]:
skipped

In [ ]:
try:
    A.reduce_scalar(agg.argmin).new()
except ValueError as e:
    exc = e
exc

In [ ]:
v = Vector.from_values([1, 3, 4, 6], [1, 1, 2, 0])
v

In [ ]:
skipped = set()
for aggname in sorted(dir(agg)):
    aggregator = getattr(agg, aggname)
    if not isinstance(aggregator, agg.Aggregator):
        continue
    try:
        with Recorder() as r:
            # The type-handing of aggregators still needs work, so hack it for now
            if aggname in {"all", "any"}:
                w = v.dup(dtype=bool)
            elif aggname in {"bitwise_all", "bitwise_any"}:
                w = v.dup(dtype="UINT64")
            else:
                w = v
            result = w.reduce(aggregator).new()
            display(Markdown(f"# {aggname}"))
            display(HTML(result._repr_html_()))
            display(Markdown(r._repr_markdown_()))
    except Exception:
        skipped.add(aggname)

In [ ]:
skipped